In [ ]:
import pandas as pd
# 读取 CSV文件
#df = pd.read_csv('C:/Python/Python Quantity/Daily_Strategy/Old/Ashare日线分钟线/沪深/stock_data_output/csi_300_all_stocks.csv')
df = pd.read_csv('C:/Python/Vanyosr_Strategy/stock_data_output/Vanyosr_stocks.csv')
# 初始化 'Signal' 和 'stop' 列，'stop'用于记录止损点
df['Signal'] = None
df['stop'] = None

# 遍历每只股票，按日期顺序查找 Selected 从 1 变为 0 的位置
for stock in df['ts_code'].unique():
    stock_data = df[df['ts_code'] == stock].copy()
    stock_data.sort_values(by='trade_date', inplace=True)  # 确保按日期排序

    # 追踪是否已经有 'B' 信号
    buy_signal_flag = False
    buy_price = None  # 记录买入价，用于止损计算

    # 找到 Selected 从1变为 0的行，并为当前交易日赋值 'B' 信号
    for i in range(1, len(stock_data)):  # 遍历从第二行开始，比较今天和昨天的数据
        # 检查今天的开盘价是否大于等于 9，若小于 9 则跳过买入信号
        if stock_data.iloc[i]['open'] < 0:
            continue  # 如果开盘价小于 9，跳过买入信号的赋值
        
        if not buy_signal_flag and stock_data.iloc[i-1]['Selected'] == 1 and stock_data.iloc[i]['Selected'] == 0:
            df.loc[stock_data.index[i], 'Signal'] = 'B'  # 当天赋值为 "B"
            buy_signal_flag = True  # 记录买入信号
            buy_price = stock_data.iloc[i]['close']  # 记录买入时的收盘价
            stop_price = buy_price * (1 - 0.04)  # 计算止损点，设为买入价格的 6%下跌幅度
            df.loc[stock_data.index[i], 'stop'] = stop_price  # 记录止损点位
            
        # 止损机制: 如果当前收盘价比买入价低 6%
        if buy_signal_flag and buy_price is not None and stock_data.iloc[i]['close'] < stop_price:
            df.loc[stock_data.index[i], 'Signal'] = 'T'  # 当天赋值为 'T'
            buy_signal_flag = False  # 重置，允许新的买入信号
            buy_price = None  # 清空买入价格
            stop_price = None # 清空止损价位
            
        # 卖出信号: Down 列小于 -0.04
        elif buy_signal_flag and stock_data.iloc[i]['Down'] < -0.04:
            df.loc[stock_data.index[i], 'Signal'] = 'S'  # 当天赋值为 'S'
            buy_signal_flag = False  # 清除买入信号，防止连续多次 'S'
            buy_price = None  # 清空买入价格

# 查看结果
print(df[['ts_code', 'trade_date', 'Selected', 'Signal', 'open', 'low', 'high', 'close', 'MACD_Hist', 'Selected', 'Down', 'stop']])

# 保存结果到新的 CSV文件
df.to_csv('C:/Python/Vanyosr_Strategy/Daily_Signal/Signal.csv', index=False)

        ts_code  trade_date  Selected Signal   open    low   high  close  \
0     000807.SZ    20240401         0   None  14.16  13.94  14.40  14.12   
1     000807.SZ    20240402         0   None  14.01  13.94  14.40  14.24   
2     000807.SZ    20240403         0   None  14.38  14.26  15.58  15.14   
3     000807.SZ    20240408         0   None  15.05  14.57  15.66  14.72   
4     000807.SZ    20240409         0   None  14.54  14.13  14.76  14.27   
...         ...         ...       ...    ...    ...    ...    ...    ...   
4195  605117.SH    20250623         0   None  50.70  50.33  52.76  52.47   
4196  605117.SH    20250624         0   None  52.58  52.00  52.80  52.63   
4197  605117.SH    20250625         0   None  52.74  51.95  53.05  52.87   
4198  605117.SH    20250626         0   None  52.76  51.27  52.76  51.36   
4199  605117.SH    20250627         0   None  51.33  51.30  52.57  52.03   

      MACD_Hist  Selected    Down  stop  
0           NaN         0     NaN  None  
1  

In [ ]:
import pandas as pd

# 读取 CSV文件
df = pd.read_csv('C:/Python/Vanyosr_Strategy/stock_data_output/Vanyosr_stocks.csv')
# 初始化 'Signal' 和 'stop' 列，'stop'用于记录止损点
df['Signal'] = None
df['stop'] = None

# 遍历每只股票，按日期顺序查找 Selected 从 1 变为 0 的位置
for stock in df['ts_code'].unique():
    stock_data = df[df['ts_code'] == stock].copy()
    stock_data.sort_values(by='trade_date', inplace=True)  # 确保按日期排序

    # 追踪是否已经有 'B' 信号
    buy_signal_flag = False
    buy_price = None  # 记录买入价，用于止损计算

    # 找到 Selected 从1变为 0的行，并为当前交易日赋值 'B' 信号
    for i in range(1, len(stock_data)):
        if stock_data.iloc[i]['open'] < 0:
            continue
        
        if not buy_signal_flag and stock_data.iloc[i-1]['Selected'] == 1 and stock_data.iloc[i]['Selected'] == 0:
            df.loc[stock_data.index[i], 'Signal'] = 'B'
            buy_signal_flag = True
            buy_price = stock_data.iloc[i]['close']
            stop_price = buy_price * (1 - 0.03)
            df.loc[stock_data.index[i], 'stop'] = stop_price
            
        if buy_signal_flag and buy_price is not None and stock_data.iloc[i]['close'] < stop_price:
            df.loc[stock_data.index[i], 'Signal'] = 'T'
            buy_signal_flag = False
            buy_price = None
            stop_price = None
            
        elif buy_signal_flag and stock_data.iloc[i]['Down'] < -0.04:
            df.loc[stock_data.index[i], 'Signal'] = 'S'
            buy_signal_flag = False
            buy_price = None

# 创建信号日志 DataFrame
signal_log = df[df['Signal'].notnull()][['ts_code', 'trade_date', 'Signal', 'open', 'close','stop','Down']]
signal_log['action'] = signal_log['Signal'].map({'B': 'buy', 'T': 'sell', 'S': 'sell'})
signal_log.sort_values(by='trade_date', inplace=True)

# 查看结果
print(df[['ts_code', 'trade_date', 'Selected', 'Signal', 'open', 'low', 'high', 'close', 'MACD_Hist', 'Selected', 'Down', 'factor_turn','stop']])
print(signal_log)

# 保存结果到新的 CSV文件
df.to_csv('C:/Python/Vanyosr_Strategy/Daily_Signal/Signal.csv', index=False)
signal_log.to_csv('C:/Python/Vanyosr_Strategy/Daily_Signal/Signal_log.csv', index=False)
print("信号日志已成功导出到 Signal_log.csv 文件中！")

        ts_code  trade_date  Selected Signal   open    low   high  close  \
0     000807.SZ    20240401         0   None  14.16  13.94  14.40  14.12   
1     000807.SZ    20240402         0   None  14.01  13.94  14.40  14.24   
2     000807.SZ    20240403         0   None  14.38  14.26  15.58  15.14   
3     000807.SZ    20240408         0   None  15.05  14.57  15.66  14.72   
4     000807.SZ    20240409         0   None  14.54  14.13  14.76  14.27   
...         ...         ...       ...    ...    ...    ...    ...    ...   
4195  605117.SH    20250623         0   None  50.70  50.33  52.76  52.47   
4196  605117.SH    20250624         0   None  52.58  52.00  52.80  52.63   
4197  605117.SH    20250625         0   None  52.74  51.95  53.05  52.87   
4198  605117.SH    20250626         0   None  52.76  51.27  52.76  51.36   
4199  605117.SH    20250627         0   None  51.33  51.30  52.57  52.03   

      MACD_Hist  Selected    Down   factor_turn  stop  
0           NaN         0     N